
# Final Project
# Big Data - Fall 2021

## By ma5416, sp6322 & lk2849

### NYPD Complaint Data Historic Public Safety
### This dataset includes all valid felony, misdemeanor, and violation crimes reported to the New York City Police Department (NYPD)
### https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i


#### Import dependencies:

In [1]:
import sys #access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features
print("pandas version: {}". format(pd.__version__))

import matplotlib #collection of functions for scientific and publication-ready visualization
import matplotlib as plt
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import scipy as sp #collection of functions for scientific computing and advance mathematics
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
print("IPython version: {}". format(IPython.__version__)) 

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

#misc libraries
import random
import time
#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)
import re
import os
import matplotlib.pyplot as plt
%matplotlib inline

Python version: 3.8.8 (default, Apr 13 2021, 12:59:45) 
[Clang 10.0.0 ]
pandas version: 1.2.4
matplotlib version: 3.3.4
NumPy version: 1.20.1
SciPy version: 1.6.2
IPython version: 7.29.0
scikit-learn version: 0.24.1
-------------------------


#### Download data file if it has not been downloaded already:

In [48]:
#url = 'https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD'
fn_src = 'https://data.cityofnewyork.us/api/views/833y-fsy8/rows.csv?accessType=DOWNLOAD'
fn_dst = 'NYPD-Shooting_Incident_Data.csv'

from six.moves import urllib

if os.path.isfile(fn_dst):
    print('File %s has already been downloaded' % fn_dst)
else:
    print('Fetching file %s[2.4GB]. This may take a while...' % fn_dst)
    urllib.request.urlretrieve(fn_src, fn_dst)
    print('File %s has been downloaded' % fn_dst)

File NYPD-Shooting_Incident_Data.csv has already been downloaded


#### Open a stream to the data file so we don't have to load the whole data set into main memory

In [49]:
from openclean.pipeline import stream
ds_full = stream(fn_dst)

In [50]:
print(f'The dataset contains {ds_full.count():,} rows.')

The dataset contains 23,585 rows.


In [51]:
ds_full.head()

,INCIDENT_KEY,OCCUR_DATE,OCCUR_TIME,BORO,PRECINCT,JURISDICTION_CODE,LOCATION_DESC,STATISTICAL_MURDER_FLAG,PERP_AGE_GROUP,PERP_SEX,PERP_RACE,VIC_AGE_GROUP,VIC_SEX,VIC_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
0,24050482,08/27/2006,05:35:00,BRONX,52,0,,true,,,,25-44,F,BLACK HISPANIC,1017541.562500000000000,255918.875000000000000,40.869058190000032,-73.879631730999961,POINT (-73.87963173099996 40.86905819000003)
1,77673979,03/11/2011,12:03:00,QUEENS,106,0,,false,,,,65+,M,WHITE,1027543.000000000000000,186095.000000000000000,40.677366895000034,-73.843920191999985,POINT (-73.84392019199998 40.677366895000034)
2,203350417,10/06/2019,01:09:00,BROOKLYN,77,0,,false,,,,18-24,F,BLACK,995325.000000000000000,185155.000000000000000,40.674885741000026,-73.960075018999987,POINT (-73.96007501899999 40.674885741000026)
3,80584527,09/04/2011,03:35:00,BRONX,40,0,,false,,,,<18,M,BLACK,1007453.000000000000000,233952.000000000000000,40.80879780500004,-73.916184131999955,POINT (-73.91618413199996 40.80879780500004)
4,90843766,05/27/2013,21:16:00,QUEENS,100,0,,false,,,,18-24,M,BLACK,1041267.375000000000000,157133.515625000000000,40.597796249000055,-73.794685537999953,POINT (-73.79468553799995 40.597796249000055)
5,92393427,09/01/2013,04:17:00,BROOKLYN,67,0,,false,,,,<18,M,BLACK,1001693.937500000000000,170112.890625000000000,40.63358818100005,-73.93715330699996,POINT (-73.93715330699996 40.63358818100005)
6,73057167,06/05/2010,21:16:00,BROOKLYN,77,0,,false,,,,<18,M,BLACK,1001506.750000000000000,184055.656250000000000,40.67185839800004,-73.93779204599997,POINT (-73.93779204599997 40.67185839800004)
7,211362213,03/20/2020,21:27:00,BROOKLYN,81,0,,false,,,,25-44,M,BLACK,1001586.375000000000000,189839.031250000000000,40.687732294000057,-73.937490124999954,POINT (-73.93749012499995 40.68773229400006)
8,137564752,07/04/2014,00:25:00,QUEENS,101,0,,false,,,,18-24,M,BLACK,1051520.375000000000000,155790.406250000000000,40.594037807000063,-73.757778691999988,POINT (-73.75777869199999 40.59403780700006)
9,147024011,10/18/2015,01:33:00,QUEENS,106,0,,false,,,,18-24,M,BLACK,1038231.312500000000000,184701.421875000000000,40.67348312300004,-73.805397634999963,POINT (-73.80539763499996 40.67348312300004)


#### Use openclean to profile the data set:

In [52]:
from openclean.profiling.column import DefaultColumnProfiler

#profiles = ds.profile(default_profiler=DefaultColumnProfiler)
profiles = ds_full.profile(default_profiler=DefaultColumnProfiler)

In [53]:
profiles.stats()

,total,empty,distinct,uniqueness,entropy
INCIDENT_KEY,23585,0,18564,0.787110,13.995572
OCCUR_DATE,23585,0,5054,0.214289,11.942664
OCCUR_TIME,23585,0,1401,0.059402,9.821678
BORO,23585,0,5,0.000212,1.976233
PRECINCT,23585,0,77,0.003265,5.575064
JURISDICTION_CODE,23585,2,3,0.000127,0.669795
LOCATION_DESC,23585,13581,39,0.003898,2.632157
STATISTICAL_MURDER_FLAG,23585,0,2,0.000085,0.703140
PERP_AGE_GROUP,23585,8295,9,0.000589,2.026719
PERP_SEX,23585,8261,3,0.000196,0.610530


In [8]:
# Print the most frequent data type for each column.

print('Schema\n------')
for col in ds_full.columns:
    p = profiles.column(col)
    print("  '{}' ({})".format(col, p['datatypes']['distinct'].most_common(1)[0][0]))

Schema
------
  'INCIDENT_KEY' (int)
  'OCCUR_DATE' (date)
  'OCCUR_TIME' (date)
  'BORO' (str)
  'PRECINCT' (int)
  'JURISDICTION_CODE' (int)
  'LOCATION_DESC' (str)
  'STATISTICAL_MURDER_FLAG' (str)
  'PERP_AGE_GROUP' (str)
  'PERP_SEX' (str)
  'PERP_RACE' (str)
  'VIC_AGE_GROUP' (str)
  'VIC_SEX' (str)
  'VIC_RACE' (str)
  'X_COORD_CD' (float)
  'Y_COORD_CD' (float)
  'Latitude' (float)
  'Longitude' (float)
  'Lon_Lat' (str)


### Field Name & Description:
    'CMPLNT_NUM' (int)                 Complaint Number
    'CMPLNT_FR_DT' (date)              Complaint From Date
    'CMPLNT_FR_TM' (date)              Complaint From Time
    'CMPLNT_TO_DT' (date)              Complaint To Date
    'CMPLNT_TO_TM' (date)              Complaint To Time
    'ADDR_PCT_CD' (int)                Code of Precinct in which the Incident Occured
    'RPT_DT' (date)                    Report Date
    'KY_CD' (int)                      "Key Code": Offense Classification Code (3 digits)
    'OFNS_DESC' (str)                  Offense Description
    'PD_CD' (int)                      PD Code of Offense. More granular than Key Code
    'PD_DESC' (str)                    PD Description of Offense.
    'CRM_ATPT_CPTD_CD' (str)           Whether Crime was Atempted or Completed (values: 'COMPLETED', 'ATTEMPTED')
    'LAW_CAT_CD' (str)                 Level of Offense (values: 'FELONY', 'VIOLATION', 'MISDEMEANOR')
    'BORO_NM' (str)                    Name of Borough in which Incident Occurred
    'LOC_OF_OCCUR_DESC' (str)          Description of where the incident occurred with respect to the premises
                                       (values:'FRONT OF', 'REAR OF', 'OUTSIDE', 'INSIDE', 'OPPOSITE OF')
    'PREM_TYP_DESC' (str)              Description of the type of premises in which the Incident Occurred
    'JURIS_DESC' (str)                 Description of Jurisdiction in which Incident Occurred
    'JURISDICTION_CODE' (int)          Jurisdiction Code
    'PARKS_NM' (str)                   Name of Park in which Incident Occurred, if Applicable
    'HADEVELOPT' (str)                 Name of NYCHA Housing Development in which Incident Occurred, if Applicable
    'HOUSING_PSA' (int)                Housing PSA
    'X_COORD_CD' (int)                 X-coordinate, New York State Plane Coordinate System
    'Y_COORD_CD' (int)                 Y-coordinate, New York State Plane Coordinate System
    'SUSP_AGE_GROUP' (int)             Age Group of Suspect
    'SUSP_RACE' (str)                  Race of Suspect
    'SUSP_SEX' (str)                   Sex of Suspect
    'TRANSIT_DISTRICT' (int)           Transit-District code
    'Latitude' (float)                 Global Latitude of Location where Incident Occurred
    'Longitude' (float)                Global Longitude of Location where Incident Occured
    'Lat_Lon' (str)                    'Latitude' and 'Longitude' together
    'PATROL_BORO' (str)                Patrol Borough
    'STATION_NAME' (str)               Station Name
    'VIC_AGE_GROUP' (int)              Age Group of Victim
    'VIC_RACE' (str)                   Race of Victim
    'VIC_SEX' (str)                    Sex of Victim
    
    
    (Note: some field descriptions were taken from https://www1.nyc.gov/assets/nypd/downloads/pdf/analysis_and_planning/incident_level_data_footnotes.pdf)
   

## Profiling & Cleaning of each field in the data set

In [9]:
profiles.minmax('INCIDENT_KEY')

,min,max
int,9953245,230611229


In [10]:
profiles.minmax('OCCUR_DATE')

,min,max
date,2006-01-01,2020-12-31


In [11]:
from datetime import datetime
#datetime.strptime('2014-12-04', '%Y-%m-%d').date()


def validate_date(sourceValue):
    dummy_date = datetime.strptime("01/01/1000", '%m/%d/%Y').date()
    
    try:
        source_date = sourceValue
        start_date = datetime.strptime("01/01/2006", '%m/%d/%Y').date()
        end_date = datetime.strptime("12/31/2020", '%m/%d/%Y').date()
    
        if source_date == '':
            source_date = dummy_date
        
        source_date = datetime.strptime(sourceValue, '%m/%d/%Y').date()
        
        if source_date >= start_date and source_date <= end_date:
            return source_date
        else:
            return dummy_date
    except:
        return dummy_date
    

ds_full = ds_full.update('OCCUR_DATE', validate_date)

In [12]:
profiles.minmax('OCCUR_TIME')

,min,max
date,2021-12-11,2021-12-11 23:59:00


In [13]:
profiles.minmax('PRECINCT')

,min,max
int,1,123


In [14]:
profiles.column('PRECINCT').get('topValues')

[('75', 1375),
 ('73', 1284),
 ('67', 1101),
 ('79', 921),
 ('44', 841),
 ('47', 818),
 ('46', 779),
 ('40', 759),
 ('42', 725),
 ('81', 722)]

In [15]:
profiles.column('LOCATION_DESC').get('topValues')

[('MULTI DWELL - PUBLIC HOUS', 4240),
 ('MULTI DWELL - APT BUILD', 2553),
 ('PVT HOUSE', 857),
 ('GROCERY/BODEGA', 574),
 ('BAR/NIGHT CLUB', 562),
 ('COMMERCIAL BLDG', 234),
 ('RESTAURANT/DINER', 188),
 ('NONE', 175),
 ('BEAUTY/NAIL SALON', 100),
 ('FAST FOOD', 98)]

In [16]:
ds_full.distinct('LOCATION_DESC')

Counter({'': 13581,
         'MULTI DWELL - PUBLIC HOUS': 4240,
         'MULTI DWELL - APT BUILD': 2553,
         'PVT HOUSE': 857,
         'GROCERY/BODEGA': 574,
         'BAR/NIGHT CLUB': 562,
         'FAST FOOD': 98,
         'COMMERCIAL BLDG': 234,
         'RESTAURANT/DINER': 188,
         'NONE': 175,
         'CLOTHING BOUTIQUE': 14,
         'SOCIAL CLUB/POLICY LOCATI': 66,
         'SMALL MERCHANT': 25,
         'LIQUOR STORE': 36,
         'HOTEL/MOTEL': 24,
         'SUPERMARKET': 19,
         'BEAUTY/NAIL SALON': 100,
         'SHOE STORE': 9,
         'DRY CLEANER/LAUNDRY': 30,
         'SCHOOL': 1,
         'HOSPITAL': 38,
         'DEPT STORE': 5,
         'GAS STATION': 53,
         'STORE UNCLASSIFIED': 35,
         'CHAIN STORE': 5,
         'DRUG STORE': 11,
         'TELECOMM. STORE': 5,
         'JEWELRY STORE': 12,
         'FACTORY/WAREHOUSE': 6,
         'CANDY STORE': 6,
         'VARIETY STORE': 11,
         'ATM': 1,
         'GYM/FITNESS FACILITY': 3,
   

In [17]:
ds_full.distinct('STATISTICAL_MURDER_FLAG')

Counter({'true': 4500, 'false': 19085})

In [18]:
def validate_crime_indicator(sourceValue):
    UNKNOWN = "3"
    
    try:
        crime_indicator_list = ["0", "1", "2"]
        
        if sourceValue in crime_indicator_list:
            return sourceValue
        else:
            return UNKNOWN
    except:
        return UNKNOWN

    
ds_full = ds_full.update('JURISDICTION_CODE', validate_crime_indicator)

In [19]:
ds_full.distinct('JURISDICTION_CODE')

Counter({'0': 19629, '2': 3900, '1': 54, '3': 2})

In [20]:
profiles.minmax('X_COORD_CD')

,min,max
float,914928.0625,1066815.375


In [21]:
profiles.minmax('Y_COORD_CD')

,min,max
float,125756.71875,271127.6875


In [22]:
profiles.column('PERP_AGE_GROUP').get('topValues')

[('18-24', 5508),
 ('25-44', 4714),
 ('UNKNOWN', 3148),
 ('<18', 1368),
 ('45-64', 495),
 ('65+', 54),
 ('1020', 1),
 ('940', 1),
 ('224', 1)]

In [23]:
ds_full.distinct('PERP_AGE_GROUP')

Counter({'': 8295,
         '18-24': 5508,
         'UNKNOWN': 3148,
         '25-44': 4714,
         '<18': 1368,
         '45-64': 495,
         '65+': 54,
         '1020': 1,
         '940': 1,
         '224': 1})

In [24]:
def validate_age_group(sourceValue):
    UNKNOWN = "UNKNOWN"
    
    try:
        values = ["25-44", "18-24", "45-64", "<18", "65+","UNKNOWN"]
        
        if sourceValue in values:
            return sourceValue
        else:
            return UNKNOWN
    except:
        return UNKNOWN

    
ds_full = ds_full.update('PERP_AGE_GROUP', validate_age_group)

In [25]:
profiles.column('PERP_RACE').get('topValues')

[('BLACK', 10025),
 ('WHITE HISPANIC', 1988),
 ('UNKNOWN', 1836),
 ('BLACK HISPANIC', 1096),
 ('WHITE', 255),
 ('ASIAN / PACIFIC ISLANDER', 122),
 ('AMERICAN INDIAN/ALASKAN NATIVE', 2)]

In [26]:
ds_full.distinct('PERP_RACE')

Counter({'': 8261,
         'BLACK': 10025,
         'WHITE HISPANIC': 1988,
         'WHITE': 255,
         'ASIAN / PACIFIC ISLANDER': 122,
         'BLACK HISPANIC': 1096,
         'UNKNOWN': 1836,
         'AMERICAN INDIAN/ALASKAN NATIVE': 2})

In [27]:
def validate_race(sourceValue):
    UNKNOWN = "UNKNOWN"
    
    try:
        values = ["UNKNOWN", "WHITE HISPANIC", "BLACK", "BLACK HISPANIC", "WHITE", "ASIAN / PACIFIC ISLANDER","AMERICAN INDIAN/ALASKAN NATIVE"]
        
        if sourceValue in values:
            return sourceValue
        else:
            return UNKNOWN
    except:
        return UNKNOWN

    
ds_full = ds_full.update('PERP_RACE', validate_race)

In [28]:
profiles.column('PERP_SEX').get('topValues')

[('M', 13490), ('U', 1499), ('F', 335)]

In [29]:
ds_full.distinct('PERP_SEX')

Counter({'': 8261, 'M': 13490, 'U': 1499, 'F': 335})

In [30]:
def validate_sex(sourceValue):
    UNKNOWN = "U"
    
    try:
        values = ["M", "F", "U"] 
        
        if sourceValue in values:
            return sourceValue
        else:
            return UNKNOWN
    except:
        return UNKNOWN

    
ds_full = ds_full.update('PERP_SEX', validate_sex)

In [31]:
profiles.minmax('Latitude')

,min,max
float,40.511586,40.910818


In [32]:
profiles.minmax('Longitude')

,min,max
float,-74.249303,-73.702046


In [33]:
profiles.minmax('Lon_Lat')

,min,max
str,POINT (-73.70204616699993 40.74174860900007),POINT (-74.24930349899995 40.511585566000065)


In [34]:
profiles.column('BORO').get('topValues')

[('BROOKLYN', 9734),
 ('BRONX', 6701),
 ('QUEENS', 3532),
 ('MANHATTAN', 2922),
 ('STATEN ISLAND', 696)]

In [35]:
ds_full.distinct('BORO')

Counter({'BRONX': 6701,
         'QUEENS': 3532,
         'BROOKLYN': 9734,
         'MANHATTAN': 2922,
         'STATEN ISLAND': 696})

In [36]:
profiles.column('VIC_AGE_GROUP').get('topValues')

[('25-44', 10303),
 ('18-24', 9003),
 ('<18', 2525),
 ('45-64', 1541),
 ('65+', 154),
 ('UNKNOWN', 59)]

In [37]:
ds_full.distinct('VIC_AGE_GROUP')

Counter({'25-44': 10303,
         '65+': 154,
         '18-24': 9003,
         '<18': 2525,
         '45-64': 1541,
         'UNKNOWN': 59})

In [38]:
ds_full = ds_full.update('VIC_AGE_GROUP', validate_age_group)

In [39]:
profiles.column('VIC_RACE').get('topValues')

[('BLACK', 16869),
 ('WHITE HISPANIC', 3450),
 ('BLACK HISPANIC', 2245),
 ('WHITE', 620),
 ('ASIAN / PACIFIC ISLANDER', 327),
 ('UNKNOWN', 65),
 ('AMERICAN INDIAN/ALASKAN NATIVE', 9)]

In [40]:
ds_full.distinct('VIC_RACE')

Counter({'BLACK HISPANIC': 2245,
         'WHITE': 620,
         'BLACK': 16869,
         'WHITE HISPANIC': 3450,
         'AMERICAN INDIAN/ALASKAN NATIVE': 9,
         'UNKNOWN': 65,
         'ASIAN / PACIFIC ISLANDER': 327})

In [41]:
ds_full = ds_full.update('VIC_RACE', validate_race)

In [42]:
profiles.column('VIC_SEX').get('topValues')

[('M', 21370), ('F', 2204), ('U', 11)]

In [43]:
ds_full.distinct('VIC_SEX')

Counter({'F': 2204, 'M': 21370, 'U': 11})

In [44]:
ds_full = ds_full.update('VIC_SEX', validate_sex)

In [45]:
SELECTED_COLUMNS = [\
    'INCIDENT_KEY',\
    'OCCUR_DATE',\
    'OCCUR_TIME',\
    'BORO',\
    'PRECINCT',\
    'JURISDICTION_CODE',\
    'LOCATION_DESC',\
    'STATISTICAL_MURDER_FLAG',\
    'PERP_AGE_GROUP',\
    'PERP_SEX',\
    'PERP_RACE',\
    'VIC_AGE_GROUP',\
    'VIC_RACE',\
    'VIC_SEX',\
    'X_COORD_CD',\
    'Y_COORD_CD',\
    'Latitude',\
    'Longitude',\
    'Lon_Lat',\
]

ds_full = ds_full.select(SELECTED_COLUMNS)

In [46]:
ds_full.head()

,INCIDENT_KEY,OCCUR_DATE,OCCUR_TIME,BORO,PRECINCT,JURISDICTION_CODE,LOCATION_DESC,STATISTICAL_MURDER_FLAG,PERP_AGE_GROUP,PERP_SEX,PERP_RACE,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
0,24050482,2006-08-27,05:35:00,BRONX,52,0,,true,UNKNOWN,U,UNKNOWN,25-44,BLACK HISPANIC,F,1017541.562500000000000,255918.875000000000000,40.869058190000032,-73.879631730999961,POINT (-73.87963173099996 40.86905819000003)
1,77673979,2011-03-11,12:03:00,QUEENS,106,0,,false,UNKNOWN,U,UNKNOWN,65+,WHITE,M,1027543.000000000000000,186095.000000000000000,40.677366895000034,-73.843920191999985,POINT (-73.84392019199998 40.677366895000034)
2,203350417,2019-10-06,01:09:00,BROOKLYN,77,0,,false,UNKNOWN,U,UNKNOWN,18-24,BLACK,F,995325.000000000000000,185155.000000000000000,40.674885741000026,-73.960075018999987,POINT (-73.96007501899999 40.674885741000026)
3,80584527,2011-09-04,03:35:00,BRONX,40,0,,false,UNKNOWN,U,UNKNOWN,<18,BLACK,M,1007453.000000000000000,233952.000000000000000,40.80879780500004,-73.916184131999955,POINT (-73.91618413199996 40.80879780500004)
4,90843766,2013-05-27,21:16:00,QUEENS,100,0,,false,UNKNOWN,U,UNKNOWN,18-24,BLACK,M,1041267.375000000000000,157133.515625000000000,40.597796249000055,-73.794685537999953,POINT (-73.79468553799995 40.597796249000055)
5,92393427,2013-09-01,04:17:00,BROOKLYN,67,0,,false,UNKNOWN,U,UNKNOWN,<18,BLACK,M,1001693.937500000000000,170112.890625000000000,40.63358818100005,-73.93715330699996,POINT (-73.93715330699996 40.63358818100005)
6,73057167,2010-06-05,21:16:00,BROOKLYN,77,0,,false,UNKNOWN,U,UNKNOWN,<18,BLACK,M,1001506.750000000000000,184055.656250000000000,40.67185839800004,-73.93779204599997,POINT (-73.93779204599997 40.67185839800004)
7,211362213,2020-03-20,21:27:00,BROOKLYN,81,0,,false,UNKNOWN,U,UNKNOWN,25-44,BLACK,M,1001586.375000000000000,189839.031250000000000,40.687732294000057,-73.937490124999954,POINT (-73.93749012499995 40.68773229400006)
8,137564752,2014-07-04,00:25:00,QUEENS,101,0,,false,UNKNOWN,U,UNKNOWN,18-24,BLACK,M,1051520.375000000000000,155790.406250000000000,40.594037807000063,-73.757778691999988,POINT (-73.75777869199999 40.59403780700006)
9,147024011,2015-10-18,01:33:00,QUEENS,106,0,,false,UNKNOWN,U,UNKNOWN,18-24,BLACK,M,1038231.312500000000000,184701.421875000000000,40.67348312300004,-73.805397634999963,POINT (-73.80539763499996 40.67348312300004)


# VALIDATING DATA AFTER CLEANING

In [47]:
data_df = pd.read_csv("NYPD-Shooting_Incident_Data.csv")

In [61]:

pd.to_numeric(data_df["JURISDICTION_CODE"])

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
23580    0.0
23581    0.0
23582    0.0
23583    0.0
23584    0.0
Name: JURISDICTION_CODE, Length: 23585, dtype: float64

In [63]:
def validate_jurisdiction_code(data,column_name):
    data = data[data[column_name].between(0,2)]
    return data

In [64]:
data_df = validate_jurisdiction_code(data_df,'JURISDICTION_CODE')

In [82]:
def validate_location_desc(data,column_name):
    data[column_name].replace(np.nan, 'UNKNOWN', inplace=True)
    return data

In [83]:
data_df = validate_location_desc(data_df,'LOCATION_DESC')

In [91]:
def validate_perp(data,column_name):
    data[column_name].replace(np.nan, 'UNKNOWN', inplace=True)
    return data

In [92]:
data_df = validate_location_desc(data_df,'PERP_AGE_GROUP')

In [93]:
data_df = validate_location_desc(data_df,'PERP_SEX')

In [94]:
data_df = validate_location_desc(data_df,'PERP_RACE')

In [95]:
data_df

,INCIDENT_KEY,OCCUR_DATE,OCCUR_TIME,BORO,PRECINCT,JURISDICTION_CODE,LOCATION_DESC,STATISTICAL_MURDER_FLAG,PERP_AGE_GROUP,PERP_SEX,PERP_RACE,VIC_AGE_GROUP,VIC_SEX,VIC_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
0,24050482,08/27/2006,05:35:00,BRONX,52,0.0,UNKNOWN,True,UNKNOWN,UNKNOWN,UNKNOWN,25-44,F,BLACK HISPANIC,1.017542e+06,255918.875000,40.869058,-73.879632,POINT (-73.87963173099996 40.86905819000003)
1,77673979,03/11/2011,12:03:00,QUEENS,106,0.0,UNKNOWN,False,UNKNOWN,UNKNOWN,UNKNOWN,65+,M,WHITE,1.027543e+06,186095.000000,40.677367,-73.843920,POINT (-73.84392019199998 40.677366895000034)
2,203350417,10/06/2019,01:09:00,BROOKLYN,77,0.0,UNKNOWN,False,UNKNOWN,UNKNOWN,UNKNOWN,18-24,F,BLACK,9.953250e+05,185155.000000,40.674886,-73.960075,POINT (-73.96007501899999 40.674885741000026)
3,80584527,09/04/2011,03:35:00,BRONX,40,0.0,UNKNOWN,False,UNKNOWN,UNKNOWN,UNKNOWN,<18,M,BLACK,1.007453e+06,233952.000000,40.808798,-73.916184,POINT (-73.91618413199996 40.80879780500004)
4,90843766,05/27/2013,21:16:00,QUEENS,100,0.0,UNKNOWN,False,UNKNOWN,UNKNOWN,UNKNOWN,18-24,M,BLACK,1.041267e+06,157133.515625,40.597796,-73.794686,POINT (-73.79468553799995 40.597796249000055)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23580,186329304,08/12/2018,19:50:00,BROOKLYN,84,0.0,UNKNOWN,False,25-44,M,BLACK,65+,M,BLACK,9.891547e+05,193164.265625,40.696875,-73.982314,POINT (-73.98231375199998 40.69687491500008)
23581,29277330,05/26/2007,04:57:00,BROOKLYN,81,0.0,UNKNOWN,False,UNKNOWN,UNKNOWN,UNKNOWN,25-44,M,BLACK,1.005218e+06,190528.000000,40.689615,-73.924393,POINT (-73.92439311199996 40.689615497000034)
23582,77443443,02/25/2011,01:12:00,BROOKLYN,81,0.0,MULTI DWELL - APT BUILD,False,18-24,M,BLACK,<18,M,BLACK,1.003999e+06,187611.031250,40.681612,-73.928798,POINT (-73.92879814299994 40.681611891000045)
23583,176027888,03/17/2018,00:46:00,BRONX,43,0.0,UNKNOWN,False,25-44,M,UNKNOWN,25-44,M,WHITE HISPANIC,1.018728e+06,242744.234375,40.832893,-73.875408,POINT (-73.87540796899998 40.83289318000004)


In [96]:
ds_full_jc = stream(data_df)
profiles = ds_full_jc.profile(default_profiler=DefaultColumnProfiler)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
INCIDENT_KEY,23583,0,18562,0.787092,13.995405
OCCUR_DATE,23583,0,5054,0.214307,11.942653
OCCUR_TIME,23583,0,1401,0.059407,9.821714
BORO,23583,0,5,0.000212,1.976157
PRECINCT,23583,0,77,0.003265,5.574937
JURISDICTION_CODE,23583,0,3,0.000127,0.669795
LOCATION_DESC,23583,0,40,0.001696,2.099604
STATISTICAL_MURDER_FLAG,23583,0,2,0.000085,0.703174
PERP_AGE_GROUP,23583,0,9,0.000382,1.837660
PERP_SEX,23583,0,4,0.000170,1.331032


In [97]:
data_df.to_csv('filtered2.csv')